In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

# 0. Install and Import dependencies

In [6]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

/Users/krishan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing_styles = mp.solutions.drawing_styles  # Drawing styles

# 1. Make some detections   

In [9]:
cap = cv2.VideoCapture(0)  # 0 for webcam

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance by making image writeable to false as we are not going to edit the image 

        # Make detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Drawing face landmarks on the screen
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, results.face_landmarks, 
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, 
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
            
#         used this line of code to change the colour of connections in the right hand
#         mp_drawing.DrawingSpec(colour=(0,0,255),thickness=2,circle_radius=2)

        # Right Hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

        # Pose Detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1704907929.741364       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-10 12:32:09.959 Python[30585:200396] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


# Capture Landmarks & Export to CSV
<img src="https://i.imgur.com/8bForKY.png">
<img src="https://i.imgur.com/AzKNp7A.png">

In [20]:
#sample landmark coordinates
results.face_landmarks.landmark[0].y 

0.4298594892024994

In [16]:
import numpy as np
import csv
import os

In [26]:
num_cords= len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_cords

501

In [27]:
landmarks = ['class']
for val in range(1, num_cords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [32]:
#Now we have class column and 501 columns for x,y,z and v coordinates
landmarks[0:10]

['class', 'x1', 'y1', 'z1', 'v1', 'x2', 'y2', 'z2', 'v2', 'x3']

In [33]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [34]:
class_name = "Happy"

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for webcam

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False # To improve performance by making image writeable to false as we are not going to edit the image 

        # Make detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        # Drawing face landmarks on the screen
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image, results.face_landmarks, 
                mp_holistic.FACEMESH_TESSELATION,
                landmark_drawing_spec=None, 
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())

        # Left Hand
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.left_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style())
            
#         used this line of code to change the colour of connections in the right hand
#         mp_drawing.DrawingSpec(colour=(0,0,255),thickness=2,circle_radius=2)

        # Right Hand
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image, results.right_hand_landmarks, 
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))

        # Pose Detection
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, 
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
        # Export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            row = pose_row+face_row

        except:
            pass

        cv2.imshow('Raw webcam feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
